<a href="https://colab.research.google.com/github/nidhiyashwanth/SesameAILabs-csm/blob/main/SesameAILabs_csm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the repository
!git clone https://github.com/SesameAILabs/csm.git
%cd csm

# Install the required packages
!pip install -r requirements.txt

# Log in to Hugging Face
from huggingface_hub import login
login()


fatal: destination path 'csm' already exists and is not an empty directory.
/content/csm
  Cloning https://github.com/SesameAILabs/silentcipher (to revision master) to /tmp/pip-install-oweju4ab/silentcipher_22d7c9ee921944e38f459b5cd668caf8
  Running command git clone --filter=blob:none --quiet https://github.com/SesameAILabs/silentcipher /tmp/pip-install-oweju4ab/silentcipher_22d7c9ee921944e38f459b5cd668caf8
  Resolved https://github.com/SesameAILabs/silentcipher to commit d46d7d0893a583d8968ab3a6626e2289faec9152
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
from generator import load_csm_1b
import torchaudio

generator = load_csm_1b(device="cuda")

audio = generator.generate(
    text="Hello from Sesame.",
    speaker=0,
    context=[],
    max_audio_length_ms=10_000,
)

torchaudio.save("audio.wav", audio.unsqueeze(0).cpu(), generator.sample_rate)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.21G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

enc_c.ckpt:   0%|          | 0.00/185k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/silentcipher/server.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.enc_c.load_state_dict(self.convert_dataparallel_to_nor

In [ ]:
from generator import Segment

In [ ]:
from google.colab import files

# Upload the audio files
uploaded = files.upload()

# List the uploaded files
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')

Saving conversational_a.wav to conversational_a.wav
Uploaded file: conversational_a.wav


In [ ]:
from generator import load_csm_1b, Segment
import torchaudio

# Initialize the generator
generator = load_csm_1b(device="cuda")

# Define speakers, transcripts, and audio paths
speakers = [0]
transcripts = [
    "Hey how are you doing."
]
audio_paths = [
    "conversational_a.wav"
]

# Function to load and resample audio
def load_audio(audio_path):
    audio_tensor, sample_rate = torchaudio.load(audio_path)
    audio_tensor = torchaudio.functional.resample(
        audio_tensor.squeeze(0), orig_freq=sample_rate, new_freq=generator.sample_rate
    )
    return audio_tensor

# Create segments
segments = [
    Segment(text=transcript, speaker=speaker, audio=load_audio(audio_path))
    for transcript, speaker, audio_path in zip(transcripts, speakers, audio_paths)
]

# Generate audio with context
audio = generator.generate(
    text="If you don't have the audio files, you can either record new audio files or use existing ones. Here's how you can add them to your Google Colab environment.",
    speaker=1,
    context=segments,
    max_audio_length_ms=50_000,
)

# Save the generated audio
torchaudio.save("audio.wav", audio.unsqueeze(0).cpu(), generator.sample_rate)


ckpt path or config path does not exist! Downloading the model from the Hugging Face Hub...


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/silentcipher/server.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.enc_c.load_state_dict(self.convert_dataparallel_to_nor